## Basic RamanomeSpec Workflow

In [12]:
pathin = "/Users/zijianleowang/Desktop/Projects_in_Cornell/CROPPS/Field Sampling/Yuan_WWTPs/JRTP Raman txt" # Data from JRTP WWTPs with process techniques of PDNA
pathout = "/Users/zijianleowang/Desktop/Projects_in_Cornell/CROPPS/Field Sampling/Yuan_WWTPs_out"
patheg = "/Users/zijianleowang/Desktop/Projects_in_Cornell/CROPPS/Field Sampling/Yuan_WWTPs/JRTP Raman txt/IFAS1_Endo/T1/Drop1_01.txt"

In [4]:
dataloop = 3 # pathin contains 2 layers of directory; 1st layer denotes experimental conditions; 2nd layers represents time points; end dir contains multiple txt files from LabSpec6

In [13]:
import pandas as pd

In [10]:
pd.read_csv(patheg,sep='\t',index_col=0,header=None)

,1,2,3,4,5,6,7,8,9,10,...,847,848,849,850,851,852,853,854,855,856
0,,,,,,,,,,,,,,,,,,,,,
NaN,403.065,404.836,406.609,408.381,410.153,411.924,413.696,415.465,417.235,419.005,...,1785.140,1786.650,1788.150,1789.660,1791.160,1792.670,1794.170,1795.670,1797.170,1798.680
1.0,1491.430,1445.890,1435.330,1418.230,1377.590,1341.770,1335.140,1298.840,1296.570,1296.900,...,227.901,245.589,219.161,226.557,227.868,228.713,229.084,229.912,223.247,227.357
2.0,1504.820,1453.540,1466.370,1383.410,1392.370,1375.560,1389.350,1317.370,1311.760,1267.010,...,214.744,227.275,229.485,225.150,216.146,228.245,208.939,221.015,225.587,225.485
3.0,1563.140,1486.510,1480.220,1433.970,1419.530,1379.370,1387.920,1374.860,1358.770,1334.860,...,267.373,260.615,271.722,256.577,279.911,259.177,273.120,271.118,277.069,255.893
4.0,2418.320,2356.620,2366.920,2334.620,2318.700,2287.520,2257.570,2282.240,2274.100,2251.190,...,1525.290,1485.740,1531.780,1508.970,1501.300,1506.320,1518.790,1498.410,1519.670,1508.690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137.0,1502.430,1467.390,1447.270,1429.200,1384.270,1356.520,1314.220,1319.270,1289.450,1281.240,...,174.803,165.291,168.477,166.048,178.168,176.690,165.371,172.785,165.212,166.073
138.0,1513.420,1473.610,1473.530,1394.860,1387.600,1385.080,1364.620,1328.300,1326.010,1300.220,...,180.441,192.527,186.779,181.996,187.076,183.721,175.678,190.579,184.401,188.996
139.0,1493.820,1465.960,1453.960,1385.320,1393.320,1385.080,1326.590,1327.820,1316.040,1270.800,...,178.562,173.743,161.907,166.048,161.758,166.380,169.119,157.801,164.744,157.653
